In [3]:
import os

import dask
import dask.dataframe as dd
import pandas as pd

In [3]:
len(os.listdir(os.path.join(os.getcwd(),'..','Data')))

22

In [5]:
df = dd.read_csv(os.path.join(os.getcwd(), '..', 'Data', '2019*.csv'))

In [6]:
df

,MATERIAL,PLNT,BATCH,UNRESTRICTED,TRANSIT/TRANSF.,IN QUALITY INSP.,RESTRICTED-USE,BLOCKED,RETURNS,STATUS_DATE
npartitions=21,,,,,,,,,,
,object,object,object,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [7]:
df.columns

Index(['MATERIAL', 'PLNT', 'BATCH', 'UNRESTRICTED', 'TRANSIT/TRANSF.',
       'IN QUALITY INSP.', 'RESTRICTED-USE', 'BLOCKED', 'RETURNS',
       'STATUS_DATE'],
      dtype='object')

In [1]:
df = df.rename(columns={'UNRESTRICTED' : "st1",
                       "TRANSIT/TRANSF." : "st2",
                       "MATERIAL" : 'sku_code',
                       "PLNT" : "sc_node_code",
                       "STATUS_DATE" : "status_date",
                       "BATCH" : "batch"})

NameError: name 'df' is not defined

In [9]:
df['st3'] = df['IN QUALITY INSP.'].astype(float) + df['RESTRICTED-USE'].astype(float)\
            + df['BLOCKED'].astype(float) + df['RETURNS'].astype(float)

In [4]:
df = df.drop(['IN_QUALITY_INSP','RESTRICTED-USE','BLOCKED','RETURNS'])

NameError: name 'df' is not defined

In [11]:
df_unpivoted =  df.map_partitions(pd.melt, id_vars=['status_date', 'sku_code','sc_node_code','batch'],
                                  var_name='stock_type', value_name='stock_qty')

In [13]:
df_unpivoted.to_csv('apollo_stock_data_*.csv',index=False)

['apollo_stock_data_00.csv',
 'apollo_stock_data_01.csv',
 'apollo_stock_data_02.csv',
 'apollo_stock_data_03.csv',
 'apollo_stock_data_04.csv',
 'apollo_stock_data_05.csv',
 'apollo_stock_data_06.csv',
 'apollo_stock_data_07.csv',
 'apollo_stock_data_08.csv',
 'apollo_stock_data_09.csv',
 'apollo_stock_data_10.csv',
 'apollo_stock_data_11.csv',
 'apollo_stock_data_12.csv',
 'apollo_stock_data_13.csv',
 'apollo_stock_data_14.csv',
 'apollo_stock_data_15.csv',
 'apollo_stock_data_16.csv',
 'apollo_stock_data_17.csv',
 'apollo_stock_data_18.csv',
 'apollo_stock_data_19.csv',
 'apollo_stock_data_20.csv']

In [15]:
forecast_dd = dd.read_csv(os.path.join(os.getcwd(),'..','Data','forecast_june.csv'),
                          dtype={'forecast_qty':float}
                         )

In [16]:
forecast_dd

,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
npartitions=1,,,,,,,,,,
,int64,int64,int64,int64,int64,object,object,object,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [17]:
dbDetails = {
    "driver": "postgresql",
    "username": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "alpha_emami"
}

In [18]:
uri = '%(driver)s://%(username)s:%(password)s@%(host)s:%(port)s/%(database)s' % dbDetails

In [20]:
calendar_dd = dd.read_sql_table('calendar_master',
                                  uri,
                                    schema='bi',
                                  index_col='for_date',
                                  divisions=['2019-06-01','2019-06-07','2019-06-14','2019-06-21','2019-06-30'],
                               ).reset_index()

In [21]:
calendar_dd

,for_date,for_month,for_year,for_day,for_day_reverse,days_in_month,week_in_month,days_week_in_month,week_in_year,days_week_in_year,day_in_week,fiscal_year_1,fiscal_year_2,quarter,fiscal_month,for_month_2,month_year,full_month,attribute_1,attribute_2,attribute_3,week_opp,days_in_week_opp,in_scope
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,
,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,int64,int64,object,object,object,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [22]:
products_dd = forecast_dd[['sku_code','sc_node_code','sales_channel_code','for_month','for_year']].drop_duplicates()

In [23]:
products_dd

,sku_code,sc_node_code,sales_channel_code,for_month,for_year
npartitions=1,,,,,
,object,object,object,int64,int64
,...,...,...,...,...


In [24]:
products_dd = dd.merge(calendar_dd[['for_date','for_month','for_year','days_in_month']], 
         products_dd,
         on=['for_month', 'for_year'])

In [25]:
products_dd

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code
npartitions=4,,,,,,,
,object,int64,int64,int64,object,object,object
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [26]:
products_dd = dd.merge(products_dd, 
         forecast_dd,
         on=['for_month', 'for_year', 'sku_code', 'sc_node_code','sales_channel_code'],
            how='left')

In [27]:
products_dd['forecast_dly'] = products_dd['forecast_qty']/products_dd['days_in_month']

In [30]:
df_unpivoted = df_unpivoted.rename(columns={'status_date' : 'for_date'})

In [31]:
final_df = products_dd.merge(df_unpivoted,
                 on=['sku_code', 'sc_node_code','for_date'],
                 how='left')

In [34]:
final_df = final_df.fillna(0)

In [35]:
%%time
final_df.to_csv('apollo_data_*.csv',index=False)

Wall time: 1min 22s


['apollo_data_00.csv',
 'apollo_data_01.csv',
 'apollo_data_02.csv',
 'apollo_data_03.csv',
 'apollo_data_04.csv',
 'apollo_data_05.csv',
 'apollo_data_06.csv',
 'apollo_data_07.csv',
 'apollo_data_08.csv',
 'apollo_data_09.csv',
 'apollo_data_10.csv',
 'apollo_data_11.csv',
 'apollo_data_12.csv',
 'apollo_data_13.csv',
 'apollo_data_14.csv',
 'apollo_data_15.csv',
 'apollo_data_16.csv',
 'apollo_data_17.csv',
 'apollo_data_18.csv',
 'apollo_data_19.csv',
 'apollo_data_20.csv']